In [1]:
import numpy as np
from tqdm import tqdm
import pandas as pd
import ast
import re
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt

In [4]:
df = pd.read_csv('mel_features_dataset.csv')

In [5]:
X = df['Mel_Features'].values
y = df['Ragas'].values

In [ ]:
#X = np.array(df['Mel_Features'].tolist())
#y = np.array(df['Ragas'].tolist())

In [6]:
print("Shape of X before reshape:", X.shape)
print("Shape of y before encode:", y.shape)

Shape of X before reshape: (174,)
Shape of y before encode: (174,)


In [7]:
le=LabelEncoder()
ti=le.fit_transform(y)
y_mel=to_categorical(ti)

In [8]:
X_train,X_test,y_train,y_test=train_test_split(X,y_mel,test_size=0.2,random_state=127)

In [9]:
num_classes = len(y_mel[0])
print(num_classes)

9


In [10]:
X_train = np.expand_dims(X_train, axis=-1)
X_test = np.expand_dims(X_test, axis=-1)
X_train_flat = np.array([x.flatten() for x in X_train])

In [11]:
print("Shape of X_train:", X_train_flat.shape)

Shape of X_train: (139, 1)


In [12]:
y_train_numerical = np.argmax(y_train, axis=1)
y_test_numerical = np.argmax(y_test, axis=1)
y_train_encoded = to_categorical(y_train_numerical, num_classes=num_classes)
y_test_encoded = to_categorical(y_test_numerical, num_classes=num_classes)

In [13]:
print(X_train[0])

['[[-49.23179  -40.900173 -35.22286  ... -80.       -80.       -80.      ]\n [-49.131256 -37.39643  -26.980406 ... -80.       -80.       -80.      ]\n [-50.548172 -39.183334 -25.958849 ... -80.       -80.       -80.      ]\n ...\n [-50.58346  -31.450312 -25.43192  ... -80.       -80.       -80.      ]\n [-50.48502  -32.110584 -25.729435 ... -80.       -80.       -80.      ]\n [-43.868637 -31.801788 -25.15377  ... -80.       -80.       -80.      ]]']


In [14]:
def convert_string_to_list(list_of_arrays, string_list):
  for i, item in enumerate(string_list):
        # Step 1: Evaluate the outermost string literal
        evaluated_list = ast.literal_eval(str(item))
        # Check if the evaluated_list is indeed a list and has at least one element
        if isinstance(evaluated_list, list) and len(evaluated_list) > 0:
            # Get the actual string containing the array data from inside the list
            evaluated_item_str = evaluated_list[0]

        # Step 2: Clean up the evaluated string
        cleaned_string = evaluated_item_str.strip('[]').replace('\n', ' ')
        cleaned_string = re.sub(r'\s+', ' ', cleaned_string).strip()

        if cleaned_string:
            temp_array = np.fromstring(cleaned_string, sep=' ')
            list_of_arrays.append(temp_array)
        else:
            print("Cleaned string was empty. Appending empty NumPy array.")
            list_of_arrays.append(np.array([]))


In [15]:
list_of_arrays = []
try:
  convert_string_to_list(list_of_arrays, X_train)
except (ValueError, SyntaxError) as e:
  print("Appending empty NumPy array due to error.")
except Exception as e:
  print("Appending empty NumPy array due to unexpected error.")

<ipython-input-14-730e6989d63d>:15: DeprecationWarning: string or file could not be read to its end due to unmatched data; this will raise a ValueError in the future.
  temp_array = np.fromstring(cleaned_string, sep=' ')


In [16]:
list_of_test_arrays = []
try:
  convert_string_to_list(list_of_test_arrays, X_test)
except (ValueError, SyntaxError) as e:
  print("Appending empty NumPy array due to error.")
except Exception as e:
  print("Appending empty NumPy array due to unexpected error.")

<ipython-input-14-730e6989d63d>:15: DeprecationWarning: string or file could not be read to its end due to unmatched data; this will raise a ValueError in the future.
  temp_array = np.fromstring(cleaned_string, sep=' ')


In [17]:
print("\n--- Final list_of_arrays status ---")
print(f"Total number of arrays parsed: {len(list_of_arrays)}")
if len(list_of_arrays) > 0:
    print(f"Shape of the first parsed array: {list_of_arrays[0].shape}")
    print(f"First parsed array content: {list_of_arrays[0]}")
    try:
        final_dataset = np.array(list_of_arrays)
        print(f"Shape of the final combined dataset: {final_dataset.shape}")
    except ValueError as e:
        print(f"Cannot combine list_of_arrays into a single NumPy array due to inconsistent shapes: {e}")


--- Final list_of_arrays status ---
Total number of arrays parsed: 139
Shape of the first parsed array: (3,)
First parsed array content: [-49.23179  -40.900173 -35.22286 ]
Shape of the final combined dataset: (139, 3)


In [18]:
print("\n--- Final list_of_test_arrays status ---")
print(f"Total number of arrays parsed: {len(list_of_test_arrays)}")
if len(list_of_test_arrays) > 0:
    print(f"Shape of the first parsed array: {list_of_test_arrays[0].shape}")
    print(f"First parsed array content: {list_of_test_arrays[0]}")
    try:
        final_test_dataset = np.array(list_of_test_arrays)
        print(f"Shape of the final combined dataset: {final_test_dataset.shape}")
    except ValueError as e:
        print(f"Cannot combine list_of_arrays into a single NumPy array due to inconsistent shapes: {e}")


--- Final list_of_test_arrays status ---
Total number of arrays parsed: 35
Shape of the first parsed array: (3,)
First parsed array content: [-45.880844 -39.540855 -34.65981 ]
Shape of the final combined dataset: (35, 3)


In [19]:
print(final_dataset.shape)

(139, 3)


In [20]:
model_mel = Sequential()
model_mel.add(Conv1D(32, 3, activation='relu', input_shape=(final_dataset.shape[0], 1)))
model_mel.add(MaxPooling1D(pool_size=2, padding='same'))
model_mel.add(Conv1D(64, 3, activation='relu'))
model_mel.add(Flatten())
model_mel.add(Dense(128, activation='relu'))
model_mel.add(Dense(64, activation='relu'))
model_mel.add(Dense(num_classes, activation='softmax'))

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [21]:
model_mel.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model_mel.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d (Conv1D)                 │ (None, 137, 32)        │           128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d (MaxPooling1D)    │ (None, 69, 32)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_1 (Conv1D)               │ (None, 67, 64)         │         6,208 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 4288)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │       548,992 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 9)              │           585 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 564,169 (2.15 MB)

 Trainable params: 564,169 (2.15 MB)

 Non-trainable params: 0 (0.00 B)

In [22]:
print(y_train_encoded.shape)
print(y_test_encoded.shape)

(139, 9)
(35, 9)


In [23]:
history_mel = model_mel.fit(final_dataset, y_train_encoded, epochs=30, batch_size=32, validation_data=(final_test_dataset, y_test_encoded))

Epoch 1/30


ValueError: Exception encountered when calling Conv1D.call().

[1mNegative dimension size caused by subtracting 3 from 1 for '{{node sequential_1/conv1d_1_2/convolution}} = Conv2D[T=DT_FLOAT, data_format="NHWC", dilations=[1, 1, 1, 1], explicit_paddings=[], padding="VALID", strides=[1, 1, 1, 1], use_cudnn_on_gpu=true](sequential_1/conv1d_1_2/convolution/ExpandDims, sequential_1/conv1d_1_2/convolution/ExpandDims_1)' with input shapes: [?,1,1,32], [1,3,32,64].[0m

Arguments received by Conv1D.call():
  • inputs=tf.Tensor(shape=(None, 1, 32), dtype=float32)